# EX_14 아이유팬이 좋아할 만한 다른 아티스트 찾기

## 목표: 한국어 데이터로 챗봇 만들기

### lubric

1. CSR matrix가 정상적으로 만들어졌다: 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다: 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다: MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.


### 학습 목표

- 추천 시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.


### 🚩keyword

✔ Recsys

### 1) 데이터 준비와 전처리

In [20]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/EXPLORATIONS/EX_14/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [21]:
# 3점 이상만 남깁니다.

ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [22]:
# ratings 컬럼의 이름을 counts로 바꿉니다.

ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [23]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [24]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

movie_file_path=os.getenv('HOME') + '/aiffel/EXPLORATIONS/EX_14/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 2) 데이터 탐색

In [25]:
# ratings에 있는 유니크한 영화 개수

ratings['movie_id'].nunique()

3628

In [26]:
# ratings에 있는 유니크한 사용자 수

ratings['user_id'].nunique()

6039

In [27]:
# 가장 인기 있는 영화 50개(인기순) 30개는 너무 적어서 50개 해봄

movie_count = ratings.groupby('movie_id')['user_id'].count()
ranking_50 = movie_count.sort_values(ascending=False).head(50)
ranking_50

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
1214    1920
3578    1798
2916    1786
2628    1783
50      1744
1259    1728
541     1722
1200    1720
1193    1680
919     1650
1221    1624
912     1623
2987    1605
2355    1599
1387    1598
1213    1595
1036    1593
1610    1587
2791    1586
34      1574
Name: user_id, dtype: int64

In [28]:
# ranking_50영화 이름으로 변환

movies.set_index('movie_id').loc[ranking_50.index]

,title,genre
movie_id,,
2858,American Beauty (1999),Comedy|Drama
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
2028,Saving Private Ryan (1998),Action|Drama|War
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
593,"Silence of the Lambs, The (1991)",Drama|Thriller
1198,Raiders of the Lost Ark (1981),Action|Adventure
1270,Back to the Future (1985),Comedy|Sci-Fi


### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [32]:
# 본인이 좋아하시는 데이터로 바꿔서 추가
my_favorite = [1270, 1, 1214, 1200, 919] #Back to the Future (1985), Toy Story (1995), Alien (1979), Aliens (1986), Wizard of Oz, The (1939)

# 'seohyun'이라는 user_id가 각 5점 평점 준 걸로 가정
my_fav_moive_list = pd.DataFrame({'user_id': ['seohyun']*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['seohyun']})['user_id'].any():  # user_id에 'seohyun'이라는 데이터가 없다면
    ratings = ratings.append(my_fav_moive_list)                 # 위에 임의로 만든 my_fav_moive_list 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,seohyun,1270,5,NaN
1,seohyun,1,5,NaN
2,seohyun,1214,5,NaN
3,seohyun,1200,5,NaN
4,seohyun,919,5,NaN


In [33]:
# indexing
# (참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용

# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [34]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 

print(user_to_idx['seohyun'])    
print(movie_to_idx[1])

6039
40


In [38]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing Fail!!
movie column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,22,5,NaN
1,6039,40,5,NaN
2,6039,193,5,NaN
3,6039,651,5,NaN


### 4) CSR matrix

In [39]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

CSR matrix = user_id 6040, movie_id 3628 

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [40]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [41]:
# Implicit AlternatingLeastSquares 모델의 선언

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [42]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)

csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [43]:
# 모델 훈련

als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [55]:
seohyun, Back_to_the_Future  = user_to_idx['seohyun'], movie_to_idx[1270]
seohyun_vector, Back_to_the_Future_vector = als_model.user_factors[seohyun], als_model.item_factors[Back_to_the_Future]

print('슝=3')

슝=3


In [57]:
seohyun_vector

array([ 0.0418265 ,  0.24288994,  1.0289947 ,  0.20313959, -0.91757447,
       -1.0454127 ,  0.09073793,  0.70119864,  0.49270615,  0.37777522,
       -0.29683772,  1.1901138 , -0.08408807, -0.04291779,  0.8238784 ,
       -0.45593077,  0.56071013, -0.637213  , -0.014367  ,  0.61817247,
        0.19847289,  0.43190452,  0.03863052, -0.59657174, -0.64204186,
        0.32050094,  0.04049874,  0.39499992, -0.47039354, -0.32927328,
       -0.11846159,  0.8862093 ,  0.2872978 ,  0.5596991 ,  1.1047785 ,
       -1.288452  , -0.20997386,  0.55942416, -0.5893386 ,  0.06093607,
        0.4228521 ,  0.52579325,  0.6884666 , -0.5379835 , -0.34410545,
       -0.49575537, -1.0778086 , -0.5685066 , -0.639825  , -0.4956583 ,
       -1.1761613 , -0.38617164,  0.10294868, -0.2718219 , -0.15610145,
        0.23070477, -0.02466656,  0.21860082,  0.29625863, -0.39508843,
       -0.19291097,  0.01290098, -0.16487423, -0.5785399 , -0.4910761 ,
       -0.08724622, -0.21572691,  0.33839896,  0.30065486, -0.48

In [58]:
Back_to_the_Future_vector

array([-0.00268473,  0.02906618,  0.04358092, -0.01468158, -0.01735455,
        0.00158028, -0.0061657 ,  0.01733554,  0.01348738,  0.02388779,
        0.01956679,  0.05757124,  0.00927005,  0.02512102,  0.01582305,
       -0.02889454,  0.01751983,  0.00297819,  0.00706867,  0.04676003,
       -0.02188571, -0.00215303, -0.03071255,  0.00534807, -0.00767788,
        0.01176814,  0.00465506,  0.02374802,  0.00287527, -0.02300715,
       -0.00783194,  0.02786229, -0.01267597,  0.02244652,  0.00163055,
        0.0056363 ,  0.00445475,  0.00614081, -0.00894829,  0.00082014,
       -0.00544362, -0.00214063,  0.0094281 , -0.01488995,  0.02018033,
       -0.00307917,  0.00656092,  0.00363316,  0.00617738, -0.01553907,
       -0.03165267, -0.02334121, -0.00151902,  0.01004385, -0.02048162,
        0.03520608,  0.01458682,  0.00682479, -0.00899305,  0.01146563,
        0.00339737,  0.01431738, -0.00706513, -0.00669967, -0.00836762,
        0.01034195, -0.01812642,  0.02005913,  0.0168563 ,  0.00

In [59]:
# seohyun과 Back_to_the_Future를 내적하는 코드

np.dot(seohyun_vector, Back_to_the_Future_vector)

0.51695144

In [64]:
# 모델이 Matrix, The에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다.

Matrix_The = movie_to_idx[2571]
Matrix_The_vector = als_model.item_factors[Matrix_The]
np.dot(seohyun_vector, Matrix_The_vector)

0.27591392

다시 훈련!

factors를 늘리거나 iterations를 늘리기

In [65]:
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [66]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [67]:
seohyun, Back_to_the_Future  = user_to_idx['seohyun'], movie_to_idx[1270]
seohyun_vector, Back_to_the_Future_vector = als_model.user_factors[seohyun], als_model.item_factors[Back_to_the_Future]

In [68]:
np.dot(seohyun_vector, Back_to_the_Future_vector)

0.7411872

0.51695144 -> 0.7411872 success!

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [69]:
movie_to_title = movies.set_index('movie_id')['title'].to_dict()
movie_to_title

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1

In [70]:
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾습니다. 
# 처음으로는 제가 좋아하는 toystory로 찾아보겠습니다.

favorite_movie = 1 #toy story
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(40, 1.0),
 (50, 0.5600084),
 (33, 0.38424897),
 (4, 0.35531524),
 (2938, 0.3382448),
 (330, 0.32582858),
 (2969, 0.3165465),
 (2598, 0.31447792),
 (1638, 0.31180647),
 (110, 0.31029776),
 (2608, 0.31001934),
 (2450, 0.3088003),
 (1825, 0.30745912),
 (3107, 0.30744147),
 (2531, 0.307409)]

In [71]:
# (영화의 id, 유사도) Tuple 로 반환하고 있습니다. 영화의 id를 다시 영화의 이름으로 매핑 시켜 주겠습니다.
#movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성 

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[movie_to_title[idx_to_movie[i[0]]] for i in similar_movie]

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Nobody Loves Me (Keiner liebt mich) (1994)',
 'Lion King, The (1994)',
 'Best Men (1997)',
 'Babyfever (1994)',
 'Walking Dead, The (1995)',
 'Groundhog Day (1993)',
 'Wisdom of Crocodiles, The (a.k.a. Immortality) (2000)',
 'Mad Love (1995)',
 'Secret Adventures of Tom Thumb, The (1993)',
 'New Rose Hotel (1998)',
 'Splendor (1999)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받기

In [79]:
# AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다. 
# filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

user = user_to_idx['seohyun']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(200, 0.5555308),
 (680, 0.33337063),
 (197, 0.30412468),
 (44, 0.28582567),
 (92, 0.27612925),
 (117, 0.27273878),
 (50, 0.26657787),
 (26, 0.23205966),
 (110, 0.22958627),
 (91, 0.22857578),
 (45, 0.2096439),
 (80, 0.1878095),
 (548, 0.18686527),
 (126, 0.18131755),
 (124, 0.17797619),
 (451, 0.17129102),
 (474, 0.17039931),
 (582, 0.17026412),
 (904, 0.166801),
 (154, 0.16461685)]

In [80]:
[movie_to_title[idx_to_movie[i[0]]] for i in movie_recommended]

['Terminator, The (1984)',
 'Blade Runner (1982)',
 'Jaws (1975)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Toy Story 2 (1999)',
 'E.T. the Extra-Terrestrial (1982)',
 'Groundhog Day (1993)',
 'Close Encounters of the Third Kind (1977)',
 'Mary Poppins (1964)',
 'Stand by Me (1986)',
 'Fantasia (1940)',
 'Shakespeare in Love (1998)',
 'Matrix, The (1999)',
 'American History X (1998)',
 'Chicken Run (2000)',
 'Beetlejuice (1988)',
 'Clockwork Orange, A (1971)',
 'As Good As It Gets (1997)']

In [85]:
# As Good As It Gets (1997) 추천에 기영한 영화의 기여 정도 확인

[movie_to_title[idx_to_movie[i[0]]] for i in movie_recommended].index('As Good As It Gets (1997)')

19

In [88]:
# lternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.

As_Good_As_It_Gets = 154
explain = als_model.explain(user, csr_data, itemid=As_Good_As_It_Gets)

In [89]:
# 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 
# 어떤 영화들이 이 추천에 얼마나 기여하고 있는 걸까요?

[(movie_to_title[idx_to_movie[i[0]]], i[1]) for i in explain[1]]

[('Aliens (1986)', 0.04468779163776469),
 ('Toy Story (1995)', 0.0445735069494987),
 ('Wizard of Oz, The (1939)', 0.039592912473356195),
 ('Back to the Future (1985)', 0.03866775707940854),
 ('Alien (1979)', -0.00289020023344912)]

'Aliens (1986)', 0.04468779163776469), 'Toy Story (1995)', 0.0445735069494987)가 가장 높게 나타나지만 그래도 높은 수치는 아닌 것 같다.

### Trial and error

전체적으로 추천 시스템이 어떻게 돌아가는지 알 수 있는 노드이다. 해보면서 알 수 있었던 건 1) 추천 시스템은 관련 데이터에 대한 이해가 다른 파트보다 제일 중요하다 2) 추천 시스템을 만드는 건 누구나 할 수 있지만 개개인의 취향을 완전히 만족시킬 수 있도록 확률을 높이는 건 가장 어려울 수 있는 분야다 정도. 기회가 된다면 정말 깊게 해보고 싶었던 분야여서 나중에 꼭 그런 기회가 온다면 잡고 싶다. 이번 노드는 어려운 게 하나도 없어서 금방 할 수 있었던 과제였다 :)